# Intro to Asteroid Mission Design

This notebook is dedicated to a hypothetical mission to one of eight asteroids marked as ERO and the best targts for mining by the Asterank database

One can visit the database at: https://www.asterank.com/

The start of this project features the heliocentric orbits of each body and the Earth being mapped in 2D and then 3D.  Preliminary mission calculations will be made to obtain an undersanding of fuel cost and travel time.

The ultimate goal is to obtain a realistic approach to how one might compute a trajectory and rendezvous with one of the asteroids selected in this project. Expanding into the elements beyond the scope of introductory astrodynamics is of interest, for example elements such as numerical methods in trajectory calculation.

## 2D Plots

Orbits for each body in question were calculated with the following equations.  Perihelion and Apihelion of each body were found on the JPL Small Body Database: https://ssd.jpl.nasa.gov/sbdb.cgi

Semimajor Axis:$$a = \frac{r_a + r_p}{2}$$

Eccentricity:$$e = \frac{r_a - r_p}{r_a + r_p}$$

Position Magnitude: $$r = \frac{a(1-e^2)}{1+ecos( \theta)}$$

X Coordinate of Ellipse:$$ x = ae + rcos(\theta)$$

Y Coordinate of Ellipse:$$ y =  rsin(\theta)$$

This process was iterated for 100 true anomaly values evenly spaced between 0 and 2$\pi$

The python code that calculated and plotted the orbit of each body is shown below:

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

def semimajor_axis(ra,rp):
	a = (ra + rp) / 2

	return a


def eccentricity(ra, rp):
	e  = (ra - rp) / (ra + rp)

	return e


def position(a, e, true_anomaly):
	r = a * (1 - e**2) / (1 + e * np.cos(true_anomaly))

	return r


def helio_orbit(rp_object, ra_object, object_name):

	theta = np.linspace(0, 2 * np.pi, 100)

	a_object = semimajor_axis(ra_object, rp_object)
	e_object = eccentricity(ra_object, rp_object)

	r_object=[]

	for tru_anom in theta:

		r_object_unit = position(a_object,e_object,tru_anom)
		r_object.append(r_object_unit)

	x = []

	for t, r in zip(theta, r_object):
		x_unit = a_object * e_object + r * np.cos(t)
		x.append(x_unit)

	y = []

	for t, r in zip(theta, r_object):
		y_unit =  r * np.sin(t)
		y.append(y_unit)

	sunx = a_object- rp_object	

	plt.plot(x,y,sunx,0,'o',mew =10, ms = 10)
	plt.title("Orbit of %s about the sun" % object_name)
	plt.xlabel("Kilometers")
	plt.ylabel("Kilometers")
	plt.show()

The 2D plots of each body's Heliocentric orbit are shown below:

<img src="images/Earth 2D.png">

<img src="images/Ryugu 2D.png">

<img src="images/ML 2D.png">

<img src="images/Nereus 2D.png">

<img src="images/Bennu 2D.png">

<img src="images/Didymos 2D.png">

<img src="images/UW158 2D.png">

<img src="images/Anteros 2D.png">

<img src="images/TC 2D.png">

## 3D Plot

Orbits for each body in question were calculated with the following equations.  Perihelion, Apihelion, Argument of Periapsis, Longitude of Ascending Node and Inclination of each body were found on the JPL Small Body Database: https://ssd.jpl.nasa.gov/sbdb.cgi

The transformation from a Perifocal to an ECI coordinate system was achieved with following transformation matrix:

\begin{equation*}
T_{P\rightarrow E}  = 
\begin{vmatrix}
cos(\Omega)cos(\omega)-sin(\Omega)sin(\omega)cos(i)& -cos(\Omega)sin(\omega)-sin(\Omega)cos(\omega)cos(i)&sin(\Omega)sin(i)\\
sin(\Omega)cos(\omega)+cos(\Omega)sin(\omega)cos(i)& -sin(\Omega)sin(\omega)+cos(\Omega)cos(\omega)cos(i)&-cos(\Omega)sin(i)\\sin(\omega)sin(i)&cos(\omega)sin(i)&cos(i)
\end{vmatrix}
\end{equation*}

The transformation from an ECI to an HCI coordinate system was achieved with following transformation matrix:

\begin{equation*}
T_{E\rightarrow H}  = 
\begin{vmatrix}
cos(23.5)& -sin(23.5)& 0
\\sin(23.5)& cos(23.5)& 0
\\0 & 0 & 1
\end{vmatrix}
\end{equation*}

Semimajor Axis:$$a = \frac{r_a + r_p}{2}$$

Eccentricity:$$e = \frac{r_a - r_p}{r_a + r_p}$$

Position Magnitude: $$r = \frac{a(1-e^2)}{1+ecos( \theta)}$$

Position Magnitude ECI:$$ r_{ECI}=T_{P\rightarrow E} * r_{Peri}$$

Position Magnitude HCI:$$ r_{HCI}=T_{E\rightarrow H} * r_{RCI}$$

This process was iterated for 100 true anomaly values evenly spaced between 0 and 2$\pi$

The python code that calculated and plotted the orbit of each body is shown below:

In [ ]:
import numpy as np 
import astrodynamics
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

def GCI2HCI(r_GCI):
	theta = 0.4101524
	rotM = np.array([[
		np.cos(theta), -np.sin(theta), 0],  # 3D rotation matrix about x-axis
		[
		np.sin(theta), np.cos(theta), 0],
		[
		0 , 0, 1]])

	r_HCI = rotM.dot(r_GCI)

	return r_HCI


def helio3D(rp_object, ra_object, o_object, i_object, w_object, object_name):

	o = o_object
	w = w_object
	i = i_object
	theta = np.linspace(0, 2 * np.pi, 100)

# transformation from perifocal to IJK
	IJK_transform = np.array([[
		np.cos(o)*np.cos(w)-np.sin(o)*np.sin(w)*np.cos(i),
		-np.cos(o)*np.sin(w)-np.sin(o)*np.cos(w)*np.cos(i),
		np.sin(o)*np.sin(i)],
		[
		np.sin(o)*np.cos(w)+np.cos(o)*np.sin(w)*np.cos(i),
		-np.sin(o)*np.sin(w)+np.cos(o)*np.cos(w)*np.cos(i),
		-np.cos(o)*np.sin(i)],
		[
		np.sin(w)*np.sin(i),
		np.cos(w)*np.sin(i),
		np.cos(i)]])

	a_object = semimajor_axis(ra_object,rp_object)
	e_object = eccentricity(ra_object, rp_object)

	r_I = []
	r_J = []
	r_K = []

	for true_anomaly in theta:

		r = position(a_object, e_object, true_anomaly)

		r_PQZ = []

		# R vector perifocal
		r_P = r * np.cos(true_anomaly) #perifocal coordinate
		r_Q = r * np.sin(true_anomaly) #perifocal coordinate

		r_PQZ.append(r_P)
		r_PQZ.append(r_Q)
		r_PQZ.append(0)

		np.transpose(np.array(r_PQZ))

		r_IJK = IJK_transform.dot(r_PQZ)
		r_IJK = GCI2HCI(r_IJK)  #transformation from GCI to HCI

		r_I.append(r_IJK[0])
		r_J.append(r_IJK[1])
		r_K.append(r_IJK[2])

def all_helio():
	e_i, e_j, e_k = astrodynamics.helio3D(147098291, 152098233, 3.0525809, 0.12487831, 5.0282936,  "Earth")
	ry_i, ry_j, ry_k = astrodynamics.helio3D(144107628.845, 211815625.124, 4.39159746, 0.102689937, 3.69014964,"162173 Ryugu")
	ml_i, ml_j, ml_k = astrodynamics.helio3D(164366172.496, 216352928.542, 1.822280819, 0.076406849461, 3.198891813, "1989 ML")
	n_i, n_j, n_k = astrodynamics.helio3D(142540510.367, 302879825.181, 5.48834299265, 0.02499001029682, 158.019386, "4660 Nereus")
	b_i, b_j, b_k = astrodynamics.helio3D(134172834.252, 202839752.882, 0.0359694906, 0.105328875, 1.155811136, "101955 Bennu")
	d_i, d_j, d_k = astrodynamics.helio3D(151587522.38, 340484753.713, 1.27773809, 0.0594860569, 5.5728363, "65803 Didymos")
	u_i, u_j, u_k = astrodynamics.helio3D(151228487.491, 333618211.448, 4.99164, 0.0797912174, 0.152780887, "2011 UW158")
	a_i, a_j, a_k = astrodynamics.helio3D(239984904.177, 268797454.074, 4.29926955, 0.15195011, 5.90567059, "1943 Anteros")
	t_i, t_j, t_k = astrodynamics.helio3D(165716219.195, 302699750.549, 1.547289591924442709,0.123698463055779792, 4.809273636806145724, "1992 TC")

	maxC = 2.9*10**8

	fig = plt.figure()
	ax = plt.axes(projection='3d')
	ax.plot3D(e_i, e_j, e_k, label ="Earth")
	ax.plot3D(ry_i, ry_j, ry_k, label="162173 Ryugu")
	ax.plot3D(ml_i, ml_j, ml_k,label="1989 ML")
	ax.plot3D(n_i, n_j, n_k, label="4660 Nereus")
	ax.plot3D(b_i, b_j, b_k, label="101955 Bennu")
	ax.plot3D(d_i, d_j, d_k, label="65803 Didymos")
	ax.plot3D(u_i, u_j, u_k, label= "2011 UW158")
	ax.plot3D(a_i, a_j, a_k, label="1943 Anteros")
	ax.plot3D(t_i, t_j, t_k, label='1992 TC')
	plt.title("Orbit of ERO and Earth about the sun")
	plt.xlabel("Kilometers")
	plt.ylabel("Kilometers")
	ax.set_xlim(-maxC, maxC); ax.set_ylim(-maxC, maxC); ax.set_zlim(-maxC, maxC);
	ax.scatter3D(0, 0, 0, s=700, c="orange")
	ax.legend()
	plt.show()


The 3D plots of each body's Heliocentric orbit are shown below:

<img src="images/3D top.png">

<img src="images/3D side.png">